Step 1 - import necessary libraries

In [33]:
import pandas as pd
import numpy as np

Step 2 - Read excel datafile & display header

In [34]:
df = pd.read_excel("C:/Users/edenibr/Documents/ML&AI/Coursera/IBM certification/applied data science capstone/Canada_postal_codes.xlsx")
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Step 3 - Ignore cells with a borough that is Not assigned

In [35]:
df = df[df['Borough']!='Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Step 4 - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [36]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
            row['Neighbourhood'] = row['Borough']

Step 5 - combine more than one neighborhood into one row with the neighborhoods separated with a comma

In [37]:
df.groupby(['Postcode'],as_index=False).agg({'Borough':'min','Neighbourhood':','.join})

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
